## URLs
In Django the `urls.py` file is used to define URL patterns for routing HTTP requests to the appropriate views. It acts as a mapping between URL paths and the corresponding view functions or class-based views that handle those requests. There are basically mainly two ways to define URL patterns in Django:
1. **Using `path()` and `re_path()` functions**: This is the traditional way of defining URL patterns in Django. You manually specify each URL pattern and map it to a view.
Example:
```python
    from django.urls import path
    from . import views
    urlpatterns = [
        path('users/', views.user_list, name='user_list'),
        path('users/<int:id>/', views.user_detail, name='user_detail'),
    ]
```
2. **Using Routers with Viewsets**: This is a more automated way of defining URL patterns, especially when using Django REST Framework. Routers automatically generate URL patterns for viewsets, which are classes that combine the logic for a set of related views.
Example:
```python
    from rest_framework.routers import DefaultRouter
    from .views import UserViewSet
    router = DefaultRouter()
    router.register(r'users', UserViewSet, basename='user')
    urlpatterns = router.urls
```

In [ ]:
# Viewset Routing Example(use for api based routing)
from rest_framework.routers import DefaultRouter
from .views import UserViewSet, ProfileViewSet
from django.urls import path, include

router = DefaultRouter()
router.register(r'users', UserViewSet, basename='user')
router.register(r'profiles', ProfileViewSet, basename='profile')

urlpatterns = [
    path('', include(router.urls)),
]

In [ ]:
# Path and Repath based routing(use for view based routing within the same app)
# ex. /apps/user_management/urls.py
from django.urls import path
from .views import get_test_view
urlpatterns += [
    path('test/', get_test_view, name='test_view'),
]

# In the main urls.py of the project, include the app urls as:
urlpatterns = [
    path('user-management/', include('apps.user_management.urls')),
]

### urls.py main components:
- **Import Statements**: Import necessary modules and functions such as `path`, `re_path`, `include`, and views.
- **URL Patterns List**: A list named `urlpatterns` that contains all the URL patterns defined using `path()`, `re_path()`, or routers.
- **Path Definitions**: Each path definition maps a URL pattern to a view function or class-based view.
- **Including Other URL Configurations**: You can include URL patterns from other apps using the `include()` function.

#### path arguments:
- `route`: A string that contains a URL pattern.
- `view`: The view function or class that will handle requests to this URL.
- `kwargs` (optional): Additional arguments to pass to the view.
- `name` (optional): A name for the URL pattern that can be used for reverse URL resolution.

#### re_path arguments:
- `route`: A regular expression string that defines the URL pattern.
- `view`: The view function or class that will handle requests to this URL.
- `kwargs` (optional): Additional arguments to pass to the view.
- `name` (optional): A name for the URL pattern that can be used for reverse URL resolution.

#### include arguments:
- `arg`: The module or URL configuration to include.
- `namespace` (optional): A string that specifies the namespace for the included URLs.


### Urls Pathing
- You can create an api folder inside your app and create a separate `urls.py` file there to manage all the api related urls. Then you can include that file in your main `urls.py` file of the app like this:
```python
    from django.urls import path, include

    urlpatterns = [
        path('api/', include('your_app.api.urls')),
    ]
```
- While the api folder urls.py contains the actual api endpoints the main urls.py in the app may just include the views for rendering templates or other non-api related views.
```python
    from django.urls import path
    from . import views

    urlpatterns = [
        path('items/', views.item_list, name='item_list'),
        path('items/<int:id>/', views.item_detail, name='item_detail'),
    ]
```
- This way you can keep your api endpoints separate from your regular views and manage them more effectively

#### Folder Structure Example:
```
your_app/
    ├── api/
    │   ├── __init__.py
    │   ├── urls.py ( api endpoints )
    │   └── views.py ( viewsets for api )
    ├── __init__.py
    ├── urls.py ( regular views )
    └── views.py ( regular views functions )
```

### Nested Routers and Relationships
Sometimes you need to expose resources that are children of other resources (e.g., Comments on a Post). 

```python
# Using drf-nested-routers (popular third-party package)
# pip install drf-nested-routers

from rest_framework_nested import routers
from .views import PostViewSet, CommentViewSet

router = routers.SimpleRouter()
router.register(r'posts', PostViewSet)

posts_router = routers.NestedSimpleRouter(router, r'posts', lookup='post')
posts_router.register(r'comments', CommentViewSet, basename='post-comments')

urlpatterns = [
    path(r'', include(router.urls)),
    path(r'', include(posts_router.urls)),
]
# Resulting URLs:
# /posts/
# /posts/{post_pk}/comments/
```

### API Versioning
It is best practice to version your API to prevent breaking changes for clients.

```python
# project/urls.py

urlpatterns = [
    path('api/v1/', include('apps.core.api.v1.urls')),
    path('api/v2/', include('apps.core.api.v2.urls')),
]
```